In [10]:
# basic packages
import pandas as pd
import numpy as np
import time
import os
from utm import *
from tqdm import tqdm, tqdm_pandas
from osgeo import osr

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

# reference transform
import coordTransform

In [11]:
                           # 预设地址和其他全局变量
feature_file_name = '1102 8am-8pm 21-11-7'
raw_data_path = 'F:/大学/第40期PRP/交通订单数据/traffic_data/gps_20161101.csv'
feature_dst_path = 'F:/大学/第40期PRP/特征提取/1_feature_analysis/' + feature_file_name + '.csv'
date = '20161102'
day_begin = '00:00:00'
day_end = '23:59:59'
#在此处设置时间窗(单位为3秒)和空间网格的边长(WGS84坐标系)
time_interval = 100
space_interval = 70
# 滞留时间阈值，超过阈值视为无效订单


# 设置时间区间 读取原数据
# 时间区间: 减少单次的处理量
time1 = f'{date} {day_begin}'
time2 = f'{date} {day_end}'
stamp1 = time.mktime(time.strptime(time1, '%Y%m%d %H:%M:%S'))
stamp2 = time.mktime(time.strptime(time2, '%Y%m%d %H:%M:%S'))

In [12]:
#导入原地理数据
df = pd.read_csv(raw_data_path, header = None) #注意我此处使用的是移动硬盘的地址
df.columns = ['driver_ID', 'order_ID', 'timestamp', 'lon', 'lat']
df.timestamp = df.timestamp + 8*3600
## 只取预设时间区间内的数据
df = df[(df['timestamp'] >= stamp1)&(df['timestamp'] < stamp2)].reset_index(drop = True)
df

,driver_ID,order_ID,timestamp,lon,lat
0,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958895,104.10348,30.71363
1,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958898,104.10348,30.71363
2,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958901,104.10348,30.71363
3,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958904,104.10348,30.71363
4,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958907,104.10348,30.71363
...,...,...,...,...,...
820385,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965343,104.04382,30.65824
820386,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965346,104.04341,30.65833
820387,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965349,104.04297,30.65841
820388,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965352,104.04254,30.65848


In [13]:
# 将空间坐标转换为WGS-84(耗时会很长)
xy = df[['lon','lat']].apply(lambda x: coordTransform.gcj02_to_wgs84(x[0],x[1])[:2], axis = 1)
df['lon'] = [x[0] for x in xy]
df['lat'] = [x[1] for x in xy]

# 再把WGS-84转换为UTM平面直角系(保留WGS-84数据)
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(4326)
# 2.Pseudo-Mercator
inp = osr.SpatialReference()
inp.ImportFromEPSG(3857)
# 3.定义坐标变换映射
transformation = osr.CoordinateTransformation(wgs84, inp)
# 4.转换原数据的坐标
xy = df[['lon','lat']].apply(lambda x: transformation.TransformPoint(x[0],x[1])[:2], axis = 1)
# 5.写入df
df['x'] = [x[0] for x in xy]
df['y'] = [x[1] for x in xy]
df

,driver_ID,order_ID,timestamp,lon,lat,x,y
0,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958895,104.100935,30.716003,1.158846e+07,3.595921e+06
1,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958898,104.100935,30.716003,1.158846e+07,3.595921e+06
2,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958901,104.100935,30.716003,1.158846e+07,3.595921e+06
3,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958904,104.100935,30.716003,1.158846e+07,3.595921e+06
4,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958907,104.100935,30.716003,1.158846e+07,3.595921e+06
...,...,...,...,...,...,...,...
820385,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965343,104.041365,30.660703,1.158183e+07,3.588763e+06
820386,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965346,104.040956,30.660793,1.158179e+07,3.588775e+06
820387,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965349,104.040517,30.660874,1.158174e+07,3.588785e+06
820388,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965352,104.040088,30.660945,1.158169e+07,3.588794e+06


In [14]:
# 时间窗划分
df['time_ID'] = df.timestamp.apply(lambda x: (x - stamp1)//time_interval)

# 空间网格划分
# 1.计算左边界和上边界，左右-x， 上下-y
left = df['x'].min()
up = df['y'].max()

# 2.生成横向和纵向索引
df['row_id'] = df['y'].apply(lambda y: (up - y)//space_interval)
df['col_id'] = df['x'].apply(lambda x: (x - left)//space_interval)
df

,driver_ID,order_ID,timestamp,lon,lat,x,y,time_ID,row_id,col_id
0,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958895,104.100935,30.716003,1.158846e+07,3.595921e+06,4.0,26.0,97.0
1,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958898,104.100935,30.716003,1.158846e+07,3.595921e+06,4.0,26.0,97.0
2,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958901,104.100935,30.716003,1.158846e+07,3.595921e+06,5.0,26.0,97.0
3,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958904,104.100935,30.716003,1.158846e+07,3.595921e+06,5.0,26.0,97.0
4,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958907,104.100935,30.716003,1.158846e+07,3.595921e+06,5.0,26.0,97.0
...,...,...,...,...,...,...,...,...,...,...
820385,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965343,104.041365,30.660703,1.158183e+07,3.588763e+06,69.0,128.0,2.0
820386,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965346,104.040956,30.660793,1.158179e+07,3.588775e+06,69.0,128.0,1.0
820387,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965349,104.040517,30.660874,1.158174e+07,3.588785e+06,69.0,128.0,1.0
820388,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965352,104.040088,30.660945,1.158169e+07,3.588794e+06,69.0,128.0,0.0


In [15]:
xy

0         (11588463.100098392, 3595921.3776360247)
1         (11588463.100098392, 3595921.3776360247)
2         (11588463.100098392, 3595921.3776360247)
3         (11588463.100098392, 3595921.3776360247)
4         (11588463.100098392, 3595921.3776360247)
                            ...                   
820385     (11581831.72581476, 3588762.8584808847)
820386    (11581786.213802898, 3588774.6215463057)
820387    (11581737.372499453, 3588785.1007280042)
820388    (11581689.642033247, 3588794.2841779334)
820389    (11581659.672144918, 3588799.5395223736)
Length: 820390, dtype: object

In [16]:
#处理空值，删除含所有含空值的行
df = df.dropna()
df


,driver_ID,order_ID,timestamp,lon,lat,x,y,time_ID,row_id,col_id
0,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958895,104.100935,30.716003,1.158846e+07,3.595921e+06,4.0,26.0,97.0
1,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958898,104.100935,30.716003,1.158846e+07,3.595921e+06,4.0,26.0,97.0
2,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958901,104.100935,30.716003,1.158846e+07,3.595921e+06,5.0,26.0,97.0
3,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958904,104.100935,30.716003,1.158846e+07,3.595921e+06,5.0,26.0,97.0
4,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958907,104.100935,30.716003,1.158846e+07,3.595921e+06,5.0,26.0,97.0
...,...,...,...,...,...,...,...,...,...,...
820385,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965343,104.041365,30.660703,1.158183e+07,3.588763e+06,69.0,128.0,2.0
820386,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965346,104.040956,30.660793,1.158179e+07,3.588775e+06,69.0,128.0,1.0
820387,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965349,104.040517,30.660874,1.158174e+07,3.588785e+06,69.0,128.0,1.0
820388,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965352,104.040088,30.660945,1.158169e+07,3.588794e+06,69.0,128.0,0.0


In [17]:
# 下面开始时空特征提取

#1. 计算瞬时速度

# 排序：先按司机排，同司机按订单排，同订单再按时间排
df = df.sort_values(by = ['driver_ID', 'order_ID', 'timestamp']).reset_index(drop = True)

# 将订单id下移一行，用于判断前后数据是否属于同一订单
df['orderFlag'] = df['order_ID'].shift(1)
df['identi'] = (df['orderFlag'] == df['order_ID']) #一个由boolean构成的列，方便后面所有shift完成了之后再删除分界行

# 将坐标，时间戳下移一行，匹配相应轨迹点
df['x1'] = df['x'].shift(1)
df['y1'] = df['y'].shift(1)
df['timestamp1'] = df['timestamp'].shift(1)

# 将不属于同一订单的轨迹点删除
df = df[df['identi'] == True]

# 计算相邻轨迹点之间的距离和相差时间
# 距离采用欧式距离
dist = np.sqrt(np.square(df['x'].values - df['x1'].values) + np.square(df['y'].values - df['y1'].values))
time = df['timestamp'].values - df['timestamp1'].values

# 计算速度
df['speed'] = dist/time

# 删除临时数据
df = df.drop(columns = ['x1', 'y1', 'orderFlag', 'timestamp1', 'identi'])
df

,driver_ID,order_ID,timestamp,lon,lat,x,y,time_ID,row_id,col_id,speed
1,gaAAqamkj@sqvfantBjjpqt8bnh5xquf,jeyqnaimc0Dusm_gzAskwmmjib61vxnb,1477960289,104.052503,30.706828,1.158307e+07,3.594733e+06,18.0,43.0,20.0,0.000000
2,gaAAqamkj@sqvfantBjjpqt8bnh5xquf,jeyqnaimc0Dusm_gzAskwmmjib61vxnb,1477960292,104.052503,30.706828,1.158307e+07,3.594733e+06,18.0,43.0,20.0,0.000000
3,gaAAqamkj@sqvfantBjjpqt8bnh5xquf,jeyqnaimc0Dusm_gzAskwmmjib61vxnb,1477960295,104.052503,30.706828,1.158307e+07,3.594733e+06,18.0,43.0,20.0,0.000000
4,gaAAqamkj@sqvfantBjjpqt8bnh5xquf,jeyqnaimc0Dusm_gzAskwmmjib61vxnb,1477960298,104.052503,30.706828,1.158307e+07,3.594733e+06,18.0,43.0,20.0,0.000000
5,gaAAqamkj@sqvfantBjjpqt8bnh5xquf,jeyqnaimc0Dusm_gzAskwmmjib61vxnb,1477960301,104.052503,30.706828,1.158307e+07,3.594733e+06,19.0,43.0,20.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
820385,vpyopffh79uszg5qsvioypuc9cc9trqf,mcrovj89cbDnof9fvppbuoteln52Euvb,1477961006,104.085023,30.680122,1.158669e+07,3.591276e+06,26.0,92.0,72.0,8.013362
820386,vpyopffh79uszg5qsvioypuc9cc9trqf,mcrovj89cbDnof9fvppbuoteln52Euvb,1477961009,104.084993,30.680072,1.158669e+07,3.591270e+06,26.0,92.0,72.0,2.425712
820387,vpyopffh79uszg5qsvioypuc9cc9trqf,mcrovj89cbDnof9fvppbuoteln52Euvb,1477961011,104.084993,30.680072,1.158669e+07,3.591270e+06,26.0,92.0,72.0,0.000000
820388,vpyopffh79uszg5qsvioypuc9cc9trqf,mcrovj89cbDnof9fvppbuoteln52Euvb,1477961012,104.084993,30.680072,1.158669e+07,3.591270e+06,26.0,92.0,72.0,0.000000


In [18]:
# 2.计算瞬时加速度
df['speed1'] = df['speed'].shift(1)
df['timestamp1'] = df['timestamp'].shift(1)
df['identi'] = df['order_ID'].shift(1)

df = df[df.identi == df.order_ID]

df['acc'] = (df.speed - df.speed1)/(df.timestamp - df.timestamp1)

df = df.drop(columns = ['speed1', 'timestamp1', 'identi'])

D:\anaconda3\envs\KeplerGl\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
df = df.reset_index(drop = True)

# 下面计算集体/网格平均特征

# 1. 网格平均速度：先求每辆车在网格中的平均速度，然后求网格中所有个体平均速度的军制
# 基于时空网格和估计id分组
orderGrouped = df.groupby(['row_id', 'col_id', 'time_ID', 'order_ID'])
# 网格在每个时刻（时间窗）的平均速度
grouped_speed = orderGrouped.speed.mean().reset_index()
grouped_speed = grouped_speed.groupby(['row_id', 'col_id', 'time_ID'])
grid_speed = grouped_speed.speed.mean()
# 去除异常值
grid_speed = grid_speed.clip(grid_speed.quantile(0.05), grid_speed.quantile(0.95))

# 2. 网格平均加速度
gridGrouped = df.groupby(['row_id', 'col_id', 'time_ID'])
grid_acc = gridGrouped.acc.mean()

# 3.网格浮动车流量
grouped_volume = orderGrouped.speed.last().reset_index() #每个时空网格中的每个order只保留一辆（用last（）来取）
grouped_volume = grouped_volume.groupby(['row_id', 'col_id', 'time_ID'])
grid_volume = grouped_volume['speed'].size()
grid_volume = grid_volume.clip(grid_volume.quantile(0.05), grid_volume.quantile(0.95))

# 4.网格车速标准差
grid_v_std = gridGrouped.speed.std(ddof=0)
# 去除异常值
grid_v_std = grid_v_std.clip(grid_v_std.quantile(0.05), grid_v_std.quantile(0.95))

# 5.网格平均停车次数
stopNum = gridGrouped.speed.agg(lambda x: (x==0).sum())
grid_stop = pd.concat((stopNum, grid_volume), axis = 1)
grid_stop['stopNum'] = stopNum.values/ grid_volume.values
grid_stop = grid_stop['stopNum']
grid_stop = grid_stop.clip(0, grid_stop.quantile(0.95))

# 下面进行数据整理
feature = pd.concat([grid_speed, grid_acc, grid_volume, grid_v_std, grid_stop], axis = 1).reset_index()
feature.columns = ['row_id','col_id', 'time_id', 'aveSpeed', 'gridAcc', 'volume', 'speedStd', 'stopNum']
feature

,row_id,col_id,time_id,aveSpeed,gridAcc,volume,speedStd,stopNum
0,0.0,7.0,61.0,7.388605,0.136826,1,0.000000,0.0
1,0.0,8.0,5.0,13.540352,0.270747,1,0.479032,0.0
2,0.0,8.0,7.0,15.295365,0.043777,1,0.000000,0.0
3,0.0,8.0,9.0,16.449861,0.883741,3,4.927091,0.0
4,0.0,8.0,10.0,15.365248,0.301097,2,0.692128,0.0
...,...,...,...,...,...,...,...,...
173483,138.0,13.0,53.0,18.027167,0.409417,1,0.000000,0.0
173484,138.0,13.0,59.0,13.641058,0.332140,2,2.057611,0.0
173485,138.0,13.0,61.0,14.233708,0.387163,1,0.000000,0.0
173486,138.0,13.0,64.0,14.965850,-0.266227,1,0.000000,0.0


In [20]:
feature.sort_values(['stopNum']).reset_index(drop=True)
feature['date'] = date

In [21]:
# 输出处理完时候的feature
feature.to_csv(feature_dst_path, index = None)
feature

,row_id,col_id,time_id,aveSpeed,gridAcc,volume,speedStd,stopNum,date
0,0.0,7.0,61.0,7.388605,0.136826,1,0.000000,0.0,20161102
1,0.0,8.0,5.0,13.540352,0.270747,1,0.479032,0.0,20161102
2,0.0,8.0,7.0,15.295365,0.043777,1,0.000000,0.0,20161102
3,0.0,8.0,9.0,16.449861,0.883741,3,4.927091,0.0,20161102
4,0.0,8.0,10.0,15.365248,0.301097,2,0.692128,0.0,20161102
...,...,...,...,...,...,...,...,...,...
173483,138.0,13.0,53.0,18.027167,0.409417,1,0.000000,0.0,20161102
173484,138.0,13.0,59.0,13.641058,0.332140,2,2.057611,0.0,20161102
173485,138.0,13.0,61.0,14.233708,0.387163,1,0.000000,0.0,20161102
173486,138.0,13.0,64.0,14.965850,-0.266227,1,0.000000,0.0,20161102
